<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;数据科学引论 - Python之道 </h1>

<h1, align=center> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;第5课 数据收集 - Python网络爬虫实践 I </h1>

# 爬虫概述
在阅读这个样例之前，建议先了解爬虫是什么，简单理解url、爬虫技术、网页html等基本概念，这可以参考链接http://python.jobbole.com/81334/

在完成本笔记本的操作之前，需要先阅读“5 爬虫环境搭建.pdf”，下载本笔记本所依赖的Python爬虫Scrapy。

# 定义爬虫的任务

## 涉及的语法
语法涉及类（面向对象）、列表list、字典dict、循环、函数、字符串操作、文件读写

## 概述
这个爬虫的任务是爬取http://quotes.toscrape.com/page/1/ 的前两页，提取每条名言的文字内容，作者和标签，最后以JSON格式保存到文件中


## 如何修改

在自己做定制时，只需要修改`__init__`和`parse`两个方法，通俗讲__init__方法决定了爬取哪些网站，parse则指明了在每一个网页上爬取哪些内容
- init_urls: 设置待爬取网站的列表和保存文件路径，其中变量self.urls是待爬取网站的列表，self.file是一个文件对象
- parse：方法内是针对每个url成功访问之后进行的页面解析
   关于如何解析具体网页，也就是选择器的使用，与网页格式十分相关，这个样例无法适用于其他网站。由于选择器的使用有很大的选择性，所以可以参考文档http://scrapy-chs.readthedocs.io/zh_CN/latest/topics/selectors.html


In [1]:
import scrapy
import time
import json
import os

class MySpider(scrapy.Spider):
    
    name = "spider"
    
    
    
    def __init__(self):
        
        self.file = open('demo1_quotes.json', 'w');
        
        #设置待爬取网站列表
        self.urls = []
        for i in range(1,3):
            self.urls.append('http://quotes.toscrape.com/page/' + str(i) )
            
#       初始化效果 效果等同
#         self.urls = [
#             'http://quotes.toscrape.com/page/1/',
#             'http://quotes.toscrape.com/page/2/',
#         ]
        
        print(self.urls)

        
    def start_requests(self):
        #self.init_urls()
        for url in self.urls:
            yield scrapy.Request(url=url, callback=self.parse)    
    

    #parse方法会在每个request收到response之后调用
    def parse(self, response):
        
        #提取名言列表
        quotes = response.css("div.quote");
        for quote in quotes:
            #提取每条名言中的作者名
            author = quote.css("small.author::text").extract_first();
            #提取名言的文字内容
            text = quote.css(".text::text").extract_first();
            #提取名言标签
            tags = quote.css(".tags .tag::text").extract();
            #构建字典对象
            item = {"author":author, "text": text, "tags":tags };
            #将字典转换成json字符串
            line = json.dumps(dict(item))
            #将每个条目写入文件
            self.file.write(line + "\n")
        #及时将内容写入文件，否则可能会出现少许延迟
        self.file.flush()
        os.fsync(self.file)
        #输出当前解析完成的网页网址，可以当做爬取进度来看待,与程序逻辑无关
        print("over: " + response.url)


# 执行爬虫任务
启动后，将执行Myspider。
这部分的代码块，如果确实非常了解scrapy的运行机制，那么可以做定制，否则不建议自行修改。

In [2]:
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(MySpider)
process.start() # 这句代码就是开始了整个爬虫过程 ，会输出一大堆信息，可以无视

2017-11-07 18:35:46 [scrapy.utils.log] INFO: Scrapy 1.3.3 started (bot: scrapybot)
2017-11-07 18:35:46 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-11-07 18:35:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2017-11-07 18:35:46 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.Redirec

['http://quotes.toscrape.com/page/1', 'http://quotes.toscrape.com/page/2']


2017-11-07 18:35:46 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET http://quotes.toscrape.com/page/1/> from <GET http://quotes.toscrape.com/page/1>
2017-11-07 18:35:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/1/> (referer: None)


over: http://quotes.toscrape.com/page/1/


2017-11-07 18:36:46 [scrapy.extensions.logstats] INFO: Crawled 1 pages (at 1 pages/min), scraped 0 items (at 0 items/min)
2017-11-07 18:36:54 [scrapy.downloadermiddlewares.redirect] DEBUG: Redirecting (301) to <GET http://quotes.toscrape.com/page/2/> from <GET http://quotes.toscrape.com/page/2>
2017-11-07 18:36:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://quotes.toscrape.com/page/2/> (referer: None)
2017-11-07 18:36:54 [scrapy.core.engine] INFO: Closing spider (finished)
2017-11-07 18:36:54 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 962,
 'downloader/request_count': 4,
 'downloader/request_method_count/GET': 4,
 'downloader/response_bytes': 6588,
 'downloader/response_count': 4,
 'downloader/response_status_count/200': 2,
 'downloader/response_status_count/301': 2,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 11, 7, 10, 36, 54, 490034),
 'log_count/DEBUG': 5,
 'log_count/INFO': 8,
 'response_received_count': 2,

over: http://quotes.toscrape.com/page/2/
